In [1]:
import pandas as pd # biblioteca que permite análise e manipulação de dados
from matminer.datasets import load_dataset # biblioteca que reúne dados de materiais e oferece diferentes ferramentas para manipulá-los

colunas = pd.read_csv("Dados.csv") 
colunas

,0,1,2
0,boltztrap_mp,pf_n,pf_p
1,citrine_thermal_conductivity,k_expt,NaN
2,ucsb_thermoelectrics,kappa [W/mK],NaN


In [2]:
datasets = [load_dataset(dataset) for dataset in colunas["0"]]

In [3]:
coluna_dadaset1 = list(colunas[["1", "2"]].iloc[0]) + ["formula", "structure"]
coluna_dadaset2 = [colunas["1"].iloc[1],"formula"]

In [4]:
datasets[0] = datasets[0][coluna_dadaset1]
datasets[1] = datasets[1][coluna_dadaset2]

In [5]:
datasets

[       pf_n    pf_p     formula  \
 0     0.865  0.0125  BaAg(PO3)3   
 1     1.050  0.6440         YSF   
 2     0.977  0.4200  Na3Sr3GaP4   
 3     0.889  0.0257    YbLiAlF6   
 4     0.719  1.5700        P2Ir   
 ...     ...     ...         ...   
 9031  1.270  1.1100       LiBeP   
 9032  1.070  0.8250        HfS3   
 9033  1.090  0.1400         ZnS   
 9034  1.560  1.0200        SnS2   
 9035  1.250  0.0693       CsAuO   
 
                                               structure  
 0     [[ 0.08245398 10.58009491 11.61923254] O, [3.1...  
 1     [[2.84699546 0.94899849 0.        ] F, [0.9489...  
 2     [[-2.8085287   7.06608376  1.25800196] Na, [1....  
 3     [[2.52287112 1.45658029 7.18290524] Li, [-2.85...  
 4     [[0.86371064 5.14343422 4.80450637] P, [4.5512...  
 ...                                                 ...  
 9031  [[0.90152298 0.90152298 2.11482411] Li, [2.704...  
 9032  [[0.90289168 4.52569433 1.31102215] S, [2.7086...  
 9033  [[-1.59315709e-08  2.2245195

In [6]:
dados_juntos = pd.merge(datasets[0], datasets[1], on= "formula")

In [7]:
from matminer.featurizers.conversions import StrToComposition # composição estequiométrica
from matminer.featurizers.composition import ElementProperty # propriedades dos elementos
# ferramentas oferecidas pelo próprio matminer

# Carregando o dataset
df = datasets[0]

stc = StrToComposition()
df = stc.featurize_dataframe(df, "formula", pbar=False)

# Calculando as propriedades dos elementos
ep_feat = ElementProperty("pymatgen", ["melting_point","thermal_conductivity"],["mean"])
ep_feat.set_n_jobs(4)
df = ep_feat.featurize_dataframe(df, col_id="composition")

# Selecionando as colunas que contém informações sobre condutividade térmica



c:\Users\guilherme220044\Anaconda3\envs\IP_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ElementProperty: 100%|██████████| 8924/8924 [1:12:09<00:00,  2.06it/s]   


In [20]:
df = df.dropna(axis=1) # retirando as linhas que possuirem algum dado faltante (NaN, Not a Number)

In [27]:
from matminer.featurizers.structure import DensityFeatures

densityf = DensityFeatures()

densityf.feature_labels() # utilizado para mostrar os dados disponiveis com esse metodo.

['density', 'vpa', 'packing fraction']

In [31]:
df = densityf.featurize_dataframe(df, "structure", True)


DensityFeatures:  25%|██▌       | 2270/8924 [13:19<1:35:25,  1.16it/s] 

In [30]:
df

,pf_n,pf_p,formula,structure,composition,PymatgenData mean melting_point,PymatgenData mean thermal_conductivity,density,vpa,packing fraction
1,1.050,0.6440,YSF,"[[2.84699546 0.94899849 0. ] F, [0.9489...","(Y, S, F)",746.963333,5.744233,4.685387,16.535411,0.587455
2,0.977,0.4200,Na3Sr3GaP4,"[[-2.8085287 7.06608376 1.25800196] Na, [1....","(Na, Sr, Ga, P)",530.429091,50.449455,3.058858,25.931418,0.700365
3,0.889,0.0257,YbLiAlF6,"[[2.52287112 1.45658029 7.18290524] Li, [-2.85...","(Yb, Li, Al, F)",311.704444,39.907356,5.047785,11.731326,0.440816
4,0.719,1.5700,P2Ir,"[[0.86371064 5.14343422 4.80450637] P, [4.5512...","(P, Ir)",1124.533333,50.157333,9.076969,15.498938,0.401825
5,1.100,0.0268,K3Sb,"[[3.49594099 2.47200358 6.05514742] K, [1.7479...","(K, Sb)",478.342500,81.000000,2.528643,39.246341,0.933697
6,1.020,0.5060,NaAlTe2,"[[1.49275972 0. 0.85230769] Na, [4.478...","(Na, Al, Te)",687.415000,95.250000,4.269131,29.675179,0.468391
7,1.010,0.0227,K2S,"[[0. 0. 0.] S, [4.57110998 3.23226287 7.917394...","(K, S)",353.806667,66.735000,1.760821,34.660663,0.898168
8,1.400,0.4370,Na2PdS2,[[4.30044524e-09 4.49277272e+00 9.62363945e+00...,"(Na, Pd, S)",669.302000,70.482000,3.492716,20.588894,0.667638
9,1.420,0.7810,NaYS2,"[[0. 0. 0.] Na, [ 0. -4.60654169 3.34...","(Na, Y, S)",736.647500,39.352500,3.174351,23.020248,0.621579
10,0.932,0.3470,Ca2BN2F,"[[0.91960831 6.79529702 6.78529997] B, [2.7588...","(Ca, B, N, F)",793.105000,71.179893,2.687949,14.206570,0.636499
